<a href="https://colab.research.google.com/github/KalpanaAK/Codsoft_intern_tasks/blob/main/nlptitleabstract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit textblob python-docx nltk pyngrok --quiet
python -m textblob.download_corpora



SyntaxError: invalid syntax (ipython-input-176877263.py, line 2)

In [2]:
!python -m textblob.download_corpora


[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.


In [3]:
!pip install streamlit textblob python-docx nltk pyngrok --quiet
!python -m textblob.download_corpora


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 70.1 MB/s eta 0:00:00
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews

In [4]:
!pip install streamlit textblob python-docx nltk pyngrok sentence-transformers transformers --quiet
!python -m textblob.download_corpora
import nltk
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
Finished.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
%%writefile app.py
import streamlit as st
from textblob import TextBlob
from transformers import T5ForConditionalGeneration, T5TokenizerFast
from sentence_transformers import SentenceTransformer, util
import nltk, re
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Preprocessing
nltk.download('punkt')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    text = text.lower().strip()
    text = re.sub(r'\s+', ' ', text)
    tokens = word_tokenize(text)
    lemmas = [lemmatizer.lemmatize(tok) for tok in tokens]
    return " ".join(lemmas)

# Load models
@st.cache_resource
def load_models():
    gen_model = T5ForConditionalGeneration.from_pretrained("t5-small")
    gen_tokenizer = T5TokenizerFast.from_pretrained("t5-small")
    embedder = SentenceTransformer("all-MiniLM-L6-v2")
    return gen_model, gen_tokenizer, embedder

GEN_MODEL, GEN_TOKENIZER, EMBEDDER = load_models()

# Title generation
def generate_titles(prompt, n=5):
    inp = f"Generate {n} academic research titles about: {prompt}"
    ids = GEN_TOKENIZER.encode(inp, return_tensors='pt')
    outs = GEN_MODEL.generate(ids, max_length=32, num_return_sequences=n, do_sample=True)
    titles = [GEN_TOKENIZER.decode(x, skip_special_tokens=True) for x in outs]
    return [re.sub(r'[^a-zA-Z0-9:.,() ]', '', t) for t in titles]

# Rank titles
def rank_titles(input_text, titles):
    emb1 = EMBEDDER.encode(input_text, convert_to_tensor=True)
    emb2 = EMBEDDER.encode(titles, convert_to_tensor=True)
    scores = util.cos_sim(emb1, emb2)[0]
    return sorted(zip(titles, scores), key=lambda x: x[1], reverse=True)

# Abstract
def generate_abstract(title, idea, words=150):
    prompt = f"Write an academic abstract (~{words} words) for a research paper titled '{title}'. Context: {idea}"
    ids = GEN_TOKENIZER.encode(prompt, return_tensors='pt')
    out = GEN_MODEL.generate(ids, max_length=256, do_sample=True)
    return GEN_TOKENIZER.decode(out[0], skip_special_tokens=True)

# Grammar correction
def fix_grammar(text):
    return str(TextBlob(text).correct())

# STREAMLIT APP UI
st.title("NLP Research Title & Abstract Generator (Google Colab Version)")

idea = st.text_area("Enter your Project Idea")
domain = st.selectbox("Select Domain", ["AI", "ML", "IoT", "Cyber Security", "Cloud", "Networking", "NLP"])
count = st.slider("Number of Titles", 3, 10, 5)

if st.button("Generate Titles"):
    pre = preprocess(idea)
    titles = generate_titles(pre + " " + domain, count)
    ranked = rank_titles(idea, titles)

    st.subheader("Ranked Titles")
    for t, s in ranked:
        st.write(f"✔ **{t}**")

    picked = st.selectbox("Select Title", [t for t, s in ranked])
    st.session_state["title"] = picked

if "title" in st.session_state:
    st.subheader("Selected Title")
    st.write(st.session_state["title"])

    word_len = st.number_input("Abstract Word Count", 80, 400, 150)

    if st.button("Generate Abstract"):
        abs_text = generate_abstract(st.session_state["title"], idea, word_len)
        corrected = fix_grammar(abs_text)

        st.subheader("Corrected Abstract")
        st.write(corrected)

        st.session_state["abstract"] = corrected


Writing app.py


In [6]:
from pyngrok import ngrok
public_url = ngrok.connect(8501)
public_url


ERROR:pyngrok.process.ngrok:t=2025-11-21T05:25:21+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-11-21T05:25:21+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2025-11-21T05:25:21+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

In [7]:
!npm install -g localtunnel


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸
added 22 packages in 3s
⠸
⠸3 packages are looking for funding
⠸  run `npm fund` for details
⠸

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸

⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸your url is: https://better-falcons-own.loca.lt

  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.170.50.243:8501


